# Import Libraries

In [ ]:
#DataFrames manipulation
import pandas as pd

#Library for text replacement
import re

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# Download the data from Shiller website

In [ ]:
#Download the Data from the Website
df = pd.read_excel('http://www.econ.yale.edu/~shiller/data/ie_data.xls', sheet_name = 'Data')

#Do some arrangements to the Dataframe
#Assign new column names to the dataframe
new_column_names = df.iloc[6]
df.columns = new_column_names

#Remove the first 8 rows from the dataframe
df = df.iloc[7:]

#Keep only needed data
df = df[['Date', 'P']]

#Convert the Date to good format
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].apply(lambda x: re.sub(r'(\d{4})\.1$', r'\g<1>.10', x))
df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y.%m')

#Change Columns names
df = df.rename(columns = {'P':'S&P 500 Index'})

#Delete last column
df = df.iloc[:-1]

# Create Tables with Performance in different cycles

In [ ]:
#Import table with Cycles
cycle_df = pd.read_csv('Cycles Table.csv')

#Add columns for Start and End of the cycle based on column Timeframe
cycle_df[['Start Year', 'End Year']] = cycle_df['Timeframe'].str.split('-', expand=True)

#Convert Start and End Year columns to date
cycle_df['Start Year'] = pd.to_datetime(cycle_df['Start Year'], format='%Y')
cycle_df['Start Year'] = cycle_df['Start Year'].apply(lambda dt: dt.replace(year=dt.year, month=1, day=1))

cycle_df['End Year'] = pd.to_datetime(cycle_df['End Year'], format='%Y')
cycle_df['End Year'] = cycle_df['End Year'].apply(lambda dt: dt.replace(year=dt.year, month=1, day=1))

In [ ]:
#Merge the cycle df with the df - to Add the Price from Start Year and from End Year
cycle_df = pd.merge(cycle_df, df, left_on='Start Year', right_on='Date', how='inner').drop(columns = 'Date').rename(columns = {'S&P 500 Index': 'Start Price'})
cycle_df = pd.merge(cycle_df, df, left_on='End Year', right_on='Date', how='inner').drop(columns = 'Date').rename(columns = {'S&P 500 Index': 'End Price'})

#Convert to Values to Float
cycle_df['End Price'] = cycle_df['End Price'].astype(float)
cycle_df['Start Price'] = cycle_df['Start Price'].astype(float)

#Calculate the Performance
cycle_df['Performance (%)'] = round((cycle_df['End Price'] / cycle_df['Start Price'] - 1), 4) * 100

#Keep only the needed columns
cycle_df = cycle_df[['Cycle Type', 'Timeframe', 'Performance (%)']]

In [ ]:
#Filter for the C-B cycle
cb_cycle_df = cycle_df[cycle_df['Cycle Type'] == 'C-B']

#Add the Average Performance and Hit Rate to C-B cycle 
cb_cycle_avg_perf = round(cb_cycle_df['Performance (%)'].mean(), 2)
cb_cycle_hit_rate = round(cb_cycle_df[cb_cycle_df['Performance (%)'] > 0]['Performance (%)'].count() / cb_cycle_df['Performance (%)'].count(), 4) * 100

cb_avg_perf_row = pd.DataFrame({'Cycle Type': ['C-B'], 'Timeframe': ['Average'], 'Performance (%)': [cb_cycle_avg_perf]})
cb_hit_rate_row = pd.DataFrame({'Cycle Type': ['C-B'], 'Timeframe': ['Hit Rate'], 'Performance (%)': [cb_cycle_hit_rate]})

cb_cycle_df = cb_cycle_df.append([cb_avg_perf_row, cb_hit_rate_row])

In [ ]:
#Filter for the C-A cycle
ca_cycle_df = cycle_df[cycle_df['Cycle Type'] == 'C-A']

#Add the Average Performance and Hit Rate to C-A cycle 
ca_cycle_avg_perf = round(ca_cycle_df['Performance (%)'].mean(), 2)
ca_cycle_hit_rate = round(ca_cycle_df[ca_cycle_df['Performance (%)'] > 0]['Performance (%)'].count() / ca_cycle_df['Performance (%)'].count(), 4) * 100

ca_avg_perf_row = pd.DataFrame({'Cycle Type': ['C-A'], 'Timeframe': ['Average'], 'Performance (%)': [ca_cycle_avg_perf]})
ca_hit_rate_row = pd.DataFrame({'Cycle Type': ['C-A'], 'Timeframe': ['Hit Rate'], 'Performance (%)': [ca_cycle_hit_rate]})

ca_cycle_df = ca_cycle_df.append([ca_avg_perf_row, ca_hit_rate_row])

In [ ]:
#Filter for the A-C cycle
ac_cycle_df = cycle_df[cycle_df['Cycle Type'] == 'A-C']

#Add the Average Performance and Hit Rate to A-C cycle 
ac_cycle_avg_perf = round(ac_cycle_df['Performance (%)'].mean(), 2)
ac_cycle_hit_rate = round(ac_cycle_df[ac_cycle_df['Performance (%)'] < 0]['Performance (%)'].count() / ac_cycle_df['Performance (%)'].count(), 4) * 100

ac_avg_perf_row = pd.DataFrame({'Cycle Type': ['A-C'], 'Timeframe': ['Average'], 'Performance (%)': [ac_cycle_avg_perf]})
ac_hit_rate_row = pd.DataFrame({'Cycle Type': ['A-C'], 'Timeframe': ['Hit Rate'], 'Performance (%)': [ac_cycle_hit_rate]})

ac_cycle_df = ac_cycle_df.append([ac_avg_perf_row, ac_hit_rate_row])

In [ ]:
#********************** Create Holoviews table object for C-B cycle **********************
cb_cycle_table = hv.Table(cb_cycle_df).opts(
    opts.Table(width=400, height=250, selectable = True, index_position = None, 
               title = 'Performance from Years of Hard Times to Years of Good Times'))

#Save the Plots
p = pn.panel(cb_cycle_table)
p.save('Performance_Hard_to_Good_Times.html', embed = True)

#********************** Create Holoviews table object for C-A cycle **********************
ca_cycle_table = hv.Table(ca_cycle_df).opts(
    opts.Table(width=400, height=250, selectable = True, index_position = None, 
               title = 'Performance from Years of Hard Times to Years of Panic'))

#Save the Plots
p = pn.panel(ca_cycle_table)
p.save('Performance_Hard_to_Panic_Times.html', embed = True)

#********************** Create Holoviews table object for C-A cycle **********************
ac_cycle_table = hv.Table(ac_cycle_df).opts(
    opts.Table(width=400, height=250, selectable = True, index_position = None, 
               title = 'Performance from Years of Panic to Years of Hard Times'))

#Save the Plots
p = pn.panel(ac_cycle_table)
p.save('Performance_Panic_to_Hard_Times.html', embed = True)

# Create Graphs with the Different Strategies

In [ ]:
#Import the data with the strategies 1,2, and 3
strategies_df = pd.read_csv('Cycle Strategies.csv')
strategies_df['Date'] = pd.to_datetime(strategies_df['Date'], format = '%m/%d/%Y')

In [ ]:
#Calculate the Returns for the S&P 500 Index
df['Return'] = df['S&P 500 Index'].pct_change().fillna(0)

In [ ]:
#Add the Returns to the Strategies dataframe
strategies_df = strategies_df.merge(df, on='Date', how='left')

#Calculate the Returns for Each strategy in Index format starting at 100
strategies_df['Strategy_1'] = (1 + strategies_df['Strategy_1']*strategies_df['Return']).cumprod()*100
strategies_df['Strategy_2'] = (1 + strategies_df['Strategy_2']*strategies_df['Return']).cumprod()*100
strategies_df['Strategy_3'] = (1 + strategies_df['Strategy_3']*strategies_df['Return']).cumprod()*100
strategies_df['Buy_and_Hold'] = (1 + strategies_df['Buy_and_Hold']*strategies_df['Return']).cumprod()*100

#Drop Unnecessary columns
strategies_df = strategies_df.drop(columns = ['S&P 500 Index', 'Return'])

#Round Decimal Places to 2
strategies_df = round(strategies_df, 2)

#Set Date as Index
strategies_df = strategies_df.set_index('Date')

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = strategies_df

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Date", "@Date{%F}"), (column, f"@{column+'{0,}'}")], formatters={'@Date': 'datetime'})  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        curve = curve.opts(xticks=10)
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Index (Base=100)', logy=True)
    source_curves.append(src)
    target_curves.append(tgt)
    
# Link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0], axes=['x','y'])  

#Overlay the source and target curves 
overlaid_plot_src = hv.Overlay(source_curves).relabel('')
overlaid_plot_tgt = hv.Overlay(target_curves)

overlaid_plot_tgt = overlaid_plot_tgt.relabel('Performance of the Different Strategies').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph = (overlaid_plot_tgt + overlaid_plot_src).cols(1)
full_graph = full_graph.opts(merge_tools=False, shared_axes=False)

In [ ]:
p = pn.panel(full_graph)
p.save('Tritch_Strategy_Backtest_Graph.html', embed = True)